<h1>Анализ базы данных с помощью SQL-запросов</h1>

**Описание проекта:** Компания запускает новый проект - приложение для чтения книг по подписке. Есть база данных о книгах, издательствах, авторах, с пользовательскими оценками и обзорами книг. Необходимо проанализировать базу данных.

**Цель проекта:** Из имеющейся базы агрегировать данные, которые помогут сформулировать ценностное предложение для нового продукта.

**Задачи проекта**

- Определить общий объём контента (посчитать, сколько книг вышло после 1 января 2000 года);
- Проранжировать книги по степении пользовательской удовлетворённости (для каждой книги посчитать количество обзоров и среднюю оценку);
- Определить издательство, которое выпустило наибольшее число книг, исключая брошюры (рассматриваем только книги, толщиной более 50 страниц);
- Определить самого успешного автора (с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками);
- Охарактеризовать самых активных пользователей (посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок).

**Описание данных**

База данных состоит из 5-ти таблиц:
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>books</b></span>, 
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>authors</b></span>, 
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>publishers</b></span>, 
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>ratings</b></span>, 
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>reviews</b></span>.

**Содержание проекта:**

1. [Подключение к базе данных](#step1)  
2. [Знакомство с таблицами базы данных](#step2)  
3. [Выполнение задач проекта](#step3) 
4. [Выводы](#step4)
5. [Рекомендации](#step5)

<h2>Подключение к базе данных<a id="step1"></a></h2>

Импортируем библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Для подключения к базе данных создадим функцию с предустановленными параметрами соединения:


In [2]:
def create_query(query):
    '''
    Функция принимает текст sql-запроса в параметре query 
    и отдаёт полученный из базы данных сформированный, в соотстветствии с sql-запросом, датафрейм.
    '''
    
    #Устанавливаем параметры:
    db_config = {'user': '------------', # имя пользователя
                'pwd': '--------------', # пароль
                'host': '--------------.mdb.yandexcloud.net',
                'port': ----, # порт подключения
                'db': '---------------'} # название базы данных

    #Формируем строку соединения с БД:
    connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                             db_config['pwd'],
                                                             db_config['host'],
                                                             db_config['port'],
                                                             db_config['db']) 
    
    # Подключаемся к БД (cохраняем коннектор):
    engine = create_engine(connection_string, connect_args={'sslmode':'require'})
    
    # Выполняем запрос и получаем результат (DataFrame), который при вызове функции необходимо сохранить в переменной:
    return pd.io.sql.read_sql(query, con = engine)

<h2>Знакомство с таблицами базы данных<a id="step2"></a></h2>

Создадим список названий таблиц:

In [3]:
db_tables_names = ['books', 'authors', 'publishers', 'ratings', 'reviews']

От заказчика нам известно, что таблицы в БД небольшого размера. Поэтому при запросе не будем ограничивать количество строк. 

Сделаем запросы к каждой таблице из списка db_tables_names в базе данных и сложим результаты в словарь:

In [4]:
db_table_samples = {}

for table in db_tables_names:
    
    #Составим текст запроса для получения таблиц:
    query = '''SELECT *
               FROM {};
            '''.format(table)   
    
    #Делаем запросы по очереди к каждой таблице из списка db_tables_names
    #и добавляем очередную таблицу в словарь:
    db_table_samples[table] = create_query(query)

Познакомимся с первыми строками каждой из таблиц и получим общую информацию об их составе:

In [5]:
for key in db_table_samples.keys():
    
    display('Таблица "{}"'.format(key))
    display(db_table_samples[key].head())
    display(db_table_samples[key].info())

'Таблица "books"'

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

'Таблица "authors"'

...

None

'Таблица "reviews"'

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

<b>Таблица</b> <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>books</b></span>

Содержит данные о книгах в количестве 1000 штук:

- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>book_id</span> — уникальный идентификатор книги;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>author_id</span> — уникальный идентификатор автора;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>title</span> — название книги;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>num_pages</span> — количество страниц в книге;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>publication_date</span> — дата публикации книги;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>publisher_id</span> — уникальный идентификатор издателя.

<b>Таблица</b> <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>authors</b></span>

Содержит данные о 636 авторах:

- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>author_id</span> — уникальный идентификатор автора;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>author</span> — имя автора книги.

<b>Таблица</b> <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>publishers</b></span>

Содержит данные о 340 издательствах:

- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>publisher_id</span> — уникальный идентификатор издательства;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>publisher</span> — название издательства;

<b>Таблица</b> <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>ratings</b></span>

Содержит данные о 6456 пользовательских оценках книг:

- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>rating_id</span> — уникальный идентификатор оценки;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>book_id</span> — уникальный идентификатор книги;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>username</span> — имя пользователя, оставившего оценку;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>rating</span> — пользовательская оценка книги.

<b>Таблица</b> <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'><b>reviews</b></span>

Содержит данные о 2793 пользовательских обзорах книг:

- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>review_id</span> — уникальный идентификатор обзора;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>book_id</span> — уникальный идентификатор книги;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>username</span> — имя пользователя, написавшего обзор;
- <span style='color: #EB5757; background-color: rgba(135,131,120,0.15)'>text</span> — текст обзора.


Состав и размерность таблиц в базе данных понятны, пропущенные значения отсутствуют, можно приступать к формированию запросов.

<h2>Выполнение задач проекта<a id="step3"></a></h2>

**Считаем, сколько книг вышло после 1 января 2000 года**

1. В таблице books отбираем строки, где значения колонки publication_date больше '2000-01-01'
2. Считаем и выводим количество уникальных идентификаторов книг.

Составляем текст sql-запроса:

In [6]:
query_1 = '''SELECT 
                COUNT(DISTINCT book_id)
            FROM 
                books
            WHERE 
                CAST(publication_date AS date) >= '2000-01-01';
        '''

In [7]:
books_count_released_after_2000_01_01 = create_query(query_1)
books_count_released_after_2000_01_01

,count
0,821


После 01.01.2000 вышла 821 книга.

**Для каждой книги считаем количество обзоров и среднюю оценку**

Подзапрос subq:
1. К таблице books присоединяем таблицу reviews по колонке book_id (используем LEFT JOIN, т.к. для нас сейчас важно, чтобы все книги вошли в итоговый результат)
2. Группируем по идентификатору книги.
3. По каждой книге считаем среднюю оценку.
4. Выводим колонки с идентификатором книги и средней оценкой.

Внешний запрос:
1. К таблице books присоединяем таблицу (результат подзапроса subq) по колонке book_id (также используем LEFT JOIN, т.к. для нас сейчас важно, чтобы все книги вошли в итоговый результат)
2. Группируем по идентификатору книги book_id и колонке среднего рейтинга из таблицы subq (результат подзапроса).
3. По каждой книге считаем количество отзывов.
4. Выводим колонки с названием книги, количеством отзывов и средней оценкой.

Составляем текст sql-запроса:

In [8]:
query_2 = ''' SELECT 
                    books.title,
                    COUNT(reviews.review_id) AS review_cnt,
                    subq.avg_rating
                FROM 
                    books
                LEFT JOIN reviews ON reviews.book_id = books.book_id
                LEFT JOIN  
                
                    (SELECT 
                        books.book_id,
                        AVG(ratings.rating) AS avg_rating
                    FROM 
                        books
                    LEFT JOIN ratings ON ratings.book_id = books.book_id
                    GROUP BY
                        books.book_id) AS subq
                        
                    ON subq.book_id = books.book_id
                GROUP BY
                    books.book_id,
                    subq.avg_rating
            '''

In [9]:
reviews_cnt_and_avg_ratings = create_query(query_2)
reviews_cnt_and_avg_ratings.head()

,title,review_cnt,avg_rating
0,The Christmas Shoes (Christmas Hope #1),2,4.000000
1,Holes (Holes #1),5,3.967742
2,The Beach House,3,4.000000
3,The Complete Calvin and Hobbes,2,4.500000
4,I Feel Bad About My Neck: And Other Thoughts o...,2,4.000000


Получили таблицу с уникальными идентификаторами книг, по каждой из которых выведено количество обзоров и средняя оценка.

**Определяем издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключаем из анализа брошюры)**

1. К таблице books присоединяем таблицу publishers (используем RIGHT JOIN, т.к. книги на протяжении своей жизни могут публиковаться в разных издательствах, и нас интересуют сейчас именно издательства)
2. В таблице books отбираем строки, где значения колонки num_pages больше 50.
3. Группируем строки по идентификатору издательства.
4. Для каждого издательства считаем количество книг.
5. Сортируем строки по убыванию количества книг.
6. Выводим первую строку таблицы с идентификатором издательства, названием издательства и количеством изданных книг.

Составляем текст sql-запроса:

In [10]:
query_3 = ''' SELECT 
                    books.publisher_id AS publisher_id,
                    publishers.publisher AS publisher,
                    COUNT(DISTINCT books.book_id) AS books_cnt
                FROM 
                    books
                RIGHT JOIN publishers ON publishers.publisher_id = books.publisher_id
                WHERE
                    books.num_pages > 50
                GROUP BY
                    books.publisher_id,
                    publishers.publisher
                ORDER BY
                    books_cnt DESC
                LIMIT 1;
            '''

In [11]:
max_books_cnt_publisher = create_query(query_3)
max_books_cnt_publisher

,publisher_id,publisher,books_cnt
0,212,Penguin Books,42


Получили данные об издательстве с самым большим количеством изданных книг, объёмом больше 50 страниц.

**Определяем автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**

Подзапрос subq:
1. В таблице ratings группируем строки по идентификатору книги. При этом в расчёт берём только те книги, у которых общее количество пользовательских оценок больше 50 штук.
2. По каждой книге считаем среднюю оценку пользователей.
3. Получаем таблицу с идентификаторами книг (с количеством оценок больше 50) и средней оценкой по каждой книге.

Внешний запрос:
1. Присоединяем к таблице books таблицу subq (результат подзапроса) по колонке book_id (используем RIGHT JOIN, т.к. нам сейчас нужны все книги из результата подзапроса) 
2. Присоединяем к таблице books (изменённой предыдущим соединением с subq) таблицу authors по колонке author_id (используем LEFT JOIN, т.к. для нас сейчас важно, чтобы в итоговый результат вошли все строки таблицы - результата предыдущего соединения)
3. Группируем строки по автору.
4. Для каждого автора находим среднее значение из средних пользовательских оценок его книг.
3. Получаем поля с именем автора и средним значением пользовательских оценок его книг.
4. Сортируем авторов по убыванию средних пользовательских оценок.
5. Ограничиваем вывод только первой строкой (с самой высокой средней оценкой пользователей)

Составляем текст sql-запроса:

In [12]:
query_4 = ''' SELECT 
                    authors.author,
                    AVG(subq.avg_rating) AS avg_books_rating
                FROM 
                    books
                RIGHT JOIN 
                
                    (SELECT 
                        ratings.book_id,
                        AVG(ratings.rating) AS avg_rating
                    FROM 
                        ratings
                    GROUP BY
                        ratings.book_id
                    HAVING
                        COUNT(ratings.rating) >= 50) AS subq
                
                    ON subq.book_id = books.book_id
                LEFT JOIN authors ON authors.author_id = books.author_id
                GROUP BY
                    authors.author
                ORDER BY
                    avg_books_rating DESC
                LIMIT 1;
            '''

In [13]:
authors_with_ratings = create_query(query_4)
authors_with_ratings.head()

,author,avg_books_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Получили имя автора J.K. Rowling/Mary GrandPré с самым высоким средним значением средних оценок его книг.

**Считаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

Подзапрос №1

1. В таблице ratings группируем строки по пользователям (username)
2. Считаем количество оценок, проставленных каждым пользователем.
3. Получаем таблицу с колонками "пользователь" и "количество оценок".

Подзапрос №2

1. Получаем список пользователей из таблицы подзапроса №1, у которых количество оценок больше 50.

Подзапрос №3

1. В таблице reviews отбираем строки с пользователями, которые есть в списке, полученном в подзапросе №2.
2. Группируем строки по этим пользователям.
3. По каждому пользователю считаем количество обзоров.
4. Получаем список с количеством обзоров каждого пользователя (согласно подзапросу №2 в пользователях у нас остались только те, кто проставил книгам более 50 оценок)

Внешний запрос:

1. Обращаемся к списку, полученному в подзапросе №3
2. Считаем и выводим среднее значение количества обзоров.

Составляем текст sql-запроса:

In [14]:
query_5 = ''' SELECT 
                    ROUND(AVG(subq_2.reviews_cnt), 0) AS avg_reviews_cnt
                FROM 
                
                    (SELECT 
                        username,
                        COUNT(review_id) AS reviews_cnt
                    FROM 
                        reviews
                    WHERE
                        username IN 
                        
                            (SELECT 
                                subq_1.username
                            FROM 
                
                                (SELECT 
                                    username,
                                    COUNT(rating_id) AS ratings_cnt
                                FROM 
                                    ratings
                                GROUP BY
                                    username) AS subq_1
                
                            WHERE
                                subq_1.ratings_cnt > 50)
                            
                    GROUP BY
                        username) AS subq_2;
            '''

In [15]:
avg_reviews_count = create_query(query_5)
avg_reviews_count

,avg_reviews_cnt
0,24.0


Получили среднее количество обзоров от пользователей, проставивших более 50 оценок.

<h2>Выводы<a id="step4"></a></h2>

1. После 01.01.2000 года всего издательствами, имеющимися в базе данных, была выпущена 821 книга.
2. Рейтинги книг и издательств можно определять по оценкам читателей и количеству книжных обзоров.
3. Наибольшее число книг, объёмом более 50 страниц, выпустило издательство Penguin Books (42 книги).
4. Самый высокий средний рейтинг (из самых часто оцениваемых книг) имеют книги автора J.K. Rowling/Mary GrandPré.
5. Самые активные пользователи (проставившие книгам более 50 оценок) в среднем пишут по 24 обзора.

<h2>Рекомендации<a id="step5"></a></h2>

1. В MVP нового продукта включить книги из ТОП-20 авторов по уровню средней пользовательской оценки.
2. Продукт должен давать пользователю возможность ставить книгам оценки и делать отзывы.
3. Продукт должен давать возможность фильтровать книги и авторов по уровню оценок и тональности отзывов (обзоров).
4. Продукт должен стимулировать пользователя ставить оценки и делать обзоры.